In [2]:
import numpy as np
import pandas as pd
# from pandas.tseries.offsets import CustomBusinessDay
# from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday

In [3]:
## ABRINDO PLANILHA GOOGLE

# bilbiotecas segurança google e acesso ao sheets
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]   #caso queira limitar o poder sobre a planilha, add ".readonly"

#def main():  
creds = None

if os.path.exists("token.json"):
  creds = Credentials.from_authorized_user_file("token.json", SCOPES)

if not creds or not creds.valid:
  if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())
  else:
    flow = InstalledAppFlow.from_client_secrets_file(
        "client_secret2.json", SCOPES
    )
    creds = flow.run_local_server(port=0)

  with open("token.json", "w") as token:
    token.write(creds.to_json())

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = "1td40BQ7k6heUOjKamm10gbb93hHYpkbIykDPu6IwJHU"
SAMPLE_RANGE_NAME = "C6 Pay!A:V"


service = build("sheets", "v4", credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()
result = (
    sheet.values()
    .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
    .execute()
)
values = result.get("values", [])

In [4]:
## INICIANDO O TRATAMENTO DOS DADOS E ASSOCIANDO A UM DATAFRAME
# convertendo lista values em dataframe
data = pd.DataFrame(values[1:],columns=values[0])

#tratamento dos valores vendidos no cartão
# copiando valores para manipular em uma lista
lista_aux = data['Valor da Venda'].copy()

#zerando vazios dentro da coluna valores
lista_aux = ["0,00" if valor == '' else valor for valor in lista_aux]

#convertendo string em número para poder manipular matematicamente
lista_aux = [float(numero.replace(',', '.')) for numero in lista_aux if numero != '']

#copiando valores da lista para a coluna respectiva no dataframe              
data['Valor da Venda'] = lista_aux

In [9]:
## VENDA REALIZADA NO MÊS
#criando variaveis para filtro (listas e inteiros)
autorizado = ['autorizado','AUTHORIZED','Autorizado']
debit = ['débito','Débito','DEBIT']
credit = ['CREDIT','crédito','Crédito']
ano = '2023'
mes = '11' 


#filtrando valores dentro do dataframe com variáveis desejadas para o relatorio
data_filt = data[data['Status'].isin(autorizado)  #filtro para transação autorizada
                  & (data['Ano Venda'] == ano)     #filtro ano da venda
                  & (data['Mês Venda'] == mes)     #filtro mes da venda
                  ]

#print(len(data_filt['Valor da Venda']))
venda_mes_debit = data_filt[data_filt['Operação'].isin(debit)]['Valor da Venda'].sum()
venda_mes_credit = data_filt[data_filt['Operação'].isin(credit)]['Valor da Venda'].sum()

print("Ano:",ano,"\nMês:",mes,"\n\nOperação: Débito","\nValor: R$",venda_mes_debit,
        "\n\nOperação: Crédito","\nValor: R$",venda_mes_credit,
        "\n\nValor Total: R$",round(venda_mes_credit+venda_mes_debit,2),"\n"
    )

Ano: 2023 
Mês: 11 

Operação: Débito 
Valor: R$ 2415.2 

Operação: Crédito 
Valor: R$ 1213.5 

Valor Total: R$ 3628.7 



In [44]:
## VENDA BASE PARA O MÊS
from datetime import date
from workalendar.america import Brazil

url = 'http://www.anbima.com.br/feriados/arqs/feriados_nacionais.xls'

feriados = pd.read_excel(url, usecols="A", parse_dates=True)

feriados.dropna(inplace=True)

for index, row in feriados.iterrows():
    if type(row["Data"]) == str:
        feriados.drop(index,axis=0,inplace=True)


lista_datas = []
for index, row in feriados.iterrows():
    lista_datas.append(row['Data']) 

print(lista_datas)

[Timestamp('2001-01-01 00:00:00'), Timestamp('2001-02-26 00:00:00'), Timestamp('2001-02-27 00:00:00'), Timestamp('2001-04-13 00:00:00'), Timestamp('2001-04-21 00:00:00'), Timestamp('2001-05-01 00:00:00'), Timestamp('2001-06-14 00:00:00'), Timestamp('2001-09-07 00:00:00'), Timestamp('2001-10-12 00:00:00'), Timestamp('2001-11-02 00:00:00'), Timestamp('2001-11-15 00:00:00'), Timestamp('2001-12-25 00:00:00'), Timestamp('2002-01-01 00:00:00'), Timestamp('2002-02-11 00:00:00'), Timestamp('2002-02-12 00:00:00'), Timestamp('2002-03-29 00:00:00'), Timestamp('2002-04-21 00:00:00'), Timestamp('2002-05-01 00:00:00'), Timestamp('2002-05-30 00:00:00'), Timestamp('2002-09-07 00:00:00'), Timestamp('2002-10-12 00:00:00'), Timestamp('2002-11-02 00:00:00'), Timestamp('2002-11-15 00:00:00'), Timestamp('2002-12-25 00:00:00'), Timestamp('2003-01-01 00:00:00'), Timestamp('2003-03-03 00:00:00'), Timestamp('2003-03-04 00:00:00'), Timestamp('2003-04-18 00:00:00'), Timestamp('2003-04-21 00:00:00'), Timestamp('20

In [ ]:
## PREVISÃO DE RECEBIMENTO PARA O MÊS
  #



## EXTRATO BANCÁRIO PARA O MÊS
  #

  #print(data['Data da Venda'])
  #proxima_data_util = data_atual + custom_bd
  # print(proxima_data_util)
  #print(data['Data de Recebimento'])

  
  #print(type(data['Data da Venda'][1]))

  # data_recebimento = [(pd.to_datetime(data['Data da Venda'][i],dayfirst=True)) + custom_bd for ]

  # data_recebimento = []
  # for i in range(0,len(data)):
    # data_recebimento.append(pd.to_datetime(data['Data da Venda'][i],dayfirst=True)+custom_bd)
  # print(data_recebimento)

In [ ]:
## MEDIA HISTÓRICA
